### Common imports

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import Output, Dropdown, SelectMultiple, HBox, VBox, Button, IntSlider, FloatRangeSlider, FloatSlider, Text, Textarea, Combobox, HBox, VBox, SelectMultiple, Tab
from IPython.display import display

from surianalytics.connectors import RESTSciriusConnector
from surianalytics.datamining import min_max_scaling

import pandas as pd
import numpy as np

In [ ]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', None)

In [ ]:
c = RESTSciriusConnector()
c.set_page_size(1000)

### Timestamp handlers

In [ ]:
from datetime import datetime, timedelta

In [ ]:
TIME_END = datetime.utcnow()
TIME_BEGINNING = TIME_END - timedelta(minutes=60)

In [ ]:
SLIDER_TIME_MINUTES = IntSlider(
    value=60,
    min=5,
    max=600,
    step=5,
    description='Minutes',
    orientation='horizontal',
    readout=True,
)

In [ ]:
# TimePicker is not in ipywidgests 7.7
# Use text boxes instead for now
TEXT_TIME_BEGINNING = Text(
    description="From: ",
    value=TIME_BEGINNING.isoformat(),
    continuous_update=True
)
TEXT_TIME_END = Text(
    description="To: ",
    value=TIME_END.isoformat(),
    continuous_update=True
)
_ = c.set_query_timeframe(from_date=TEXT_TIME_BEGINNING.value, to_date=TEXT_TIME_END.value)

In [ ]:
def handler_update_timeframe(args):
    c.set_query_timeframe(from_date=TEXT_TIME_BEGINNING.value, to_date=TEXT_TIME_END.value)

In [ ]:
BUTTON_UPDATE_TIME = Button(description="Set time")
BUTTON_UPDATE_TIME.on_click(handler_update_timeframe)

In [ ]:
def handler_reset_timeframe(args):
    time_to = datetime.utcnow()
    time_from = time_to - timedelta(minutes=SLIDER_TIME_MINUTES.value)
    TEXT_TIME_END.value = time_to.isoformat()
    TEXT_TIME_BEGINNING.value = time_from.isoformat()
    c.set_query_delta(minutes=SLIDER_TIME_MINUTES.value)

In [ ]:
BUTTON_RESET_TIME = Button(description="Generate time")
BUTTON_RESET_TIME.on_click(handler_reset_timeframe)

In [ ]:
import pickle
PKL_TIME = "./time.pkl"

In [ ]:
def handler_pickle_time_save(args):
    with open(PKL_TIME, "wb") as handle:
        pickle.dump((TEXT_TIME_BEGINNING.value, TEXT_TIME_END.value), handle)

In [ ]:
BUTTON_SAVE_TIME = Button(description="Dump time")
BUTTON_SAVE_TIME.on_click(handler_pickle_time_save)

In [ ]:
def handler_pickle_time_load(args):
    handle = open(PKL_TIME, "rb")
    times = pickle.load(handle)
    TEXT_TIME_BEGINNING.value = times[0]
    TEXT_TIME_END.value = times[1]
    handler_update_timeframe(args)

In [ ]:
BUTTON_LOAD_TIME = Button(description="Sync time")
BUTTON_LOAD_TIME.on_click(handler_pickle_time_load)

In [ ]:
BOX_TIME = VBox([
    HBox([SLIDER_TIME_MINUTES, BUTTON_RESET_TIME]), 
    HBox([
        VBox([TEXT_TIME_BEGINNING, TEXT_TIME_END]), 
        BUTTON_UPDATE_TIME,
        VBox([BUTTON_SAVE_TIME, BUTTON_LOAD_TIME])
    ])
])

### Query handler

In [ ]:
TEXT_QUERY = Textarea(
    description='Query filter:',
    value="*",
    continuous_update=True
)

In [ ]:
import pickle
PKL_QUERY = "./query.pkl"

In [ ]:
def handler_pickle_query_save(args):
    with open(PKL_QUERY, "wb") as handle:
        pickle.dump(TEXT_QUERY.value, handle)

In [ ]:
BUTTON_SAVE_QUERY = Button(description="Dump query")
BUTTON_SAVE_QUERY.on_click(handler_pickle_query_save)

In [ ]:
def handler_pickle_query_load(args):
    handle = open(PKL_QUERY, "rb")
    TEXT_QUERY.value = pickle.load(handle)

In [ ]:
BUTTON_LOAD_QUERY = Button(description="Sync query")
BUTTON_LOAD_QUERY.on_click(handler_pickle_query_load)

In [ ]:
BOX_QUERY = HBox([
    TEXT_QUERY, 
    VBox([BUTTON_SAVE_QUERY, BUTTON_LOAD_QUERY])
])

### Unique values

In [ ]:
SELECT_FIELD = Combobox(options=c.get_unique_fields(), value="flow_id")

In [ ]:
DF = pd.DataFrame()
OUTPUT_UNIQUE = Output()
def handler_pull_data(args):
    global DF
    DF = pd.DataFrame(c.get_eve_unique_values(counts="yes", field=SELECT_FIELD.value, qfilter=TEXT_QUERY.value))
    OUTPUT_UNIQUE.clear_output()
    with OUTPUT_UNIQUE:
        if len(DF) == 0:
            print("no data")
        else:
            display(DF.sort_values(by="key"))

In [ ]:
BUTTON_PULL_UNIQUE = Button(description="Pull unique values")
BUTTON_PULL_UNIQUE.on_click(handler_pull_data)

In [ ]:
def handler_show_simple_uniq(args):
    OUTPUT_UNIQUE.clear_output()
    with OUTPUT_UNIQUE:
        print("\n".join(sorted(list(DF.key.astype(str).unique()))))

In [ ]:
BUTTON_SHOW_SIMPLE = Button(description="Show simple values")
BUTTON_SHOW_SIMPLE.on_click(handler_show_simple_uniq)

In [ ]:
BOX_UNIQUE = VBox([
    HBox([
        SELECT_FIELD,
        VBox([BUTTON_PULL_UNIQUE, BUTTON_SHOW_SIMPLE]),
    ]),
    OUTPUT_UNIQUE
])

### Hunting section

In [ ]:
tab = Tab(children=[BOX_TIME, BOX_QUERY, BOX_UNIQUE])
tab.set_title(0, "Timepicker")
tab.set_title(1, "Query filter")
tab.set_title(2, "Analytics")

In [ ]:
tab